In [ ]:
# 7-1
!mkdir chap7
%cd ./chap7

In [2]:
# 7-3
import random
import glob
import json
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel
import pytorch_lightning as pl

# 日本語の事前学習モデル
MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

In [3]:
# 7-4（説明コメント付き：BERTを用いたマルチラベル文分類の最小実装）
# --------------------------------------------------------------------------------
# 目的：
#  - 事前学習済み BERT（エンコーダ本体：BertModel）の出力（最終層隠れ状態）から
#    「[PAD] を除いたトークン平均（mean pooling）」で文ベクトルを作り、
#    線形層でラベル数（num_labels）次元に写像して **マルチラベル**分類を行う。
#
# 理論メモ（マルチラベル vs マルチクラス）：
#  - マルチクラス（排他的1クラス）では softmax + CrossEntropyLoss を用いるが、
#  - マルチラベル（複数ラベル同時に1の可能性）では **各ラベルを独立なベルヌーイ**として扱い、
#    出力は raw logits（活性前）→ **BCEWithLogitsLoss**（内部で sigmoid + BCE を数値安定に計算）を用いる。
#  - 教師ラベルの形：shape [B, C]、各要素は {0,1}（float型推奨）。しきい値 0.5 などで陽性判定を後段で行う。
#
# プーリング設計（mean pooling の意図）：
#  - [CLS] ベクトル単独の利用に比べ、文全体の情報を平均で取り入れやすい。短文～中程度の長さで堅実なベースライン。
#  - attention_mask により [PAD] 位置は平均から除外（分母は非PADトークン数）。
#  - 分母ゼロ（全PAD）対策：学習データ生成で通常発生しない想定だが、実務では clamp などで保険をかけると安全。
#
# そのほか設計の注意：
#  - token_type_ids（= segment ids）は文対（sentence pair）で区別に使うが、単文では 0 固定でもよい。
#  - 出力は慣例的には transformers の `SequenceClassifierOutput` を使うが、
#    ここでは attributes にアクセスできる簡易オブジェクトを返す（互換性が必要なら差し替え推奨）。
#  - クラス不均衡が強い場合は BCEWithLogitsLoss の `pos_weight` を利用すると勾配が安定する。
# --------------------------------------------------------------------------------


class BertForSequenceClassificationMultiLabel(torch.nn.Module):

    def __init__(self, model_name, num_labels):
        super().__init__()
        # BertModel のロード（分類ヘッドなしの本体のみ）。
        # - 入力：input_ids, attention_mask, token_type_ids
        # - 出力：last_hidden_state（[B, L, H]）など
        self.bert = BertModel.from_pretrained(model_name)

        # 文ベクトル（プーリング後の [B, H]）から各ラベルのロジット [B, C] を直線写像する層。
        # - H: 隠れ次元（bert.config.hidden_size）、C: ラベル数（num_labels）
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(
        self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None
    ):
        # 1) BERT でトークン列を符号化し、最終層の隠れ状態を得る。
        #    last_hidden_state の形は [B, L, H]（B:バッチ、L:系列長、H:隠れ次元）
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        last_hidden_state = bert_output.last_hidden_state

        # 2) [PAD] 以外の位置のみで平均プーリングして文ベクトルを作る。
        #    - attention_mask は [B, L] の {0,1}。unsqueeze(-1) で [B, L, 1] に拡張し、
        #      last_hidden_state（float）にマスク乗算→ PAD 位置を 0 化。
        #    - 分母は非PADトークン数（attention_mask.sum(dim=1)）。
        #      典型データでは 0 になることはないが、実運用では clamp_min(1e-9) などで安全策も検討。
        averaged_hidden_state = (last_hidden_state * attention_mask.unsqueeze(-1)).sum(
            1
        ) / attention_mask.sum(1, keepdim=True)
        # 参考（安全策の例、実装切替の際に利用）：
        # denom = attention_mask.sum(1, keepdim=True).clamp_min(1e-9).float()
        # masked = last_hidden_state * attention_mask.unsqueeze(-1).float()
        # averaged_hidden_state = masked.sum(1) / denom

        # 3) 線形層でラベル数 C 次元のロジットへ（活性はかけない：BCEWithLogitsLoss を使うため）
        scores = self.linear(averaged_hidden_state)  # shape: [B, C]

        # 4) 出力を dict 形式で用意（logits を必須、labels が来ていれば loss も計算）
        output = {"logits": scores}

        # 5) 教師ラベルが与えられた場合は損失を計算して返す。
        #    - BCEWithLogitsLoss は内部で sigmoid を組み合わせた数値安定版。
        #    - labels 形状は [B, C]、型は float（{0.0, 1.0}）を想定。
        #    - クラス不均衡が強ければ pos_weight（shape [C]）の指定を検討。
        if labels is not None:
            loss = torch.nn.BCEWithLogitsLoss()(scores, labels.float())
            output["loss"] = loss

        # 6) 呼び出し側が `output.logits` / `output.loss` で参照できるよう簡易オブジェクト化。
        #    transformers の標準 `SequenceClassifierOutput` を使う場合は差し替え可。
        output = type("bert_output", (object,), output)

        return output

In [4]:
# 7-5（説明コメント付き：マルチラベル文分類モデルのロードとデバイス配置）
# -----------------------------------------------------------------------------
# 目的：
#  - 日本語BERTと同一語彙のトークナイザをロードし、
#  - 7-4で定義した「BertForSequenceClassificationMultiLabel」を初期化、
#  - 環境に応じた最適デバイス（MPS/CUDA/CPU）へ安全に配置する。
#
# 理論メモ：
#  - 本モデルは出力が各ラベルの「ロジット」（活性前）で、マルチラベル想定（独立ベルヌーイ）。
#    学習時は BCEWithLogitsLoss（sigmoid + BCE の安定版）を用いる。
#  - num_labels=2 の場合、2つのラベルそれぞれを {0,1} で同時に予測する（softmaxではない）。
# -----------------------------------------------------------------------------

import torch
from transformers import BertJapaneseTokenizer

# --- MODEL_NAME が未定義でも動くように保険を入れる ---
try:
    MODEL_NAME
except NameError:
    MODEL_NAME = (
        "tohoku-nlp/bert-base-japanese-whole-word-masking"  # 東北大版日本語BERT（WWM）
    )


# --- デバイス選択：Mac(MPS) → CUDA → CPU の順で自動選択 ---
def get_best_device() -> torch.device:
    # Apple Silicon + macOS Metal
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")
    # NVIDIA GPU（Linux/Windowsなど）
    if torch.cuda.is_available():
        return torch.device("cuda")
    # フォールバック：CPU
    return torch.device("cpu")


device = get_best_device()
print(f"[info] using device = {device}")

# --- トークナイザのロード（モデルと同じ語彙でないとIDがずれて壊れる） ---
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# --- マルチラベル分類モデルの構築 ---
# num_labels=2：2つのラベルを独立に予測（例：{スポーツ, IT} の属否を同時に判定 など）
bert_scml = BertForSequenceClassificationMultiLabel(MODEL_NAME, num_labels=2)

# --- 最適デバイスへ配置（.cuda()固定はMacで失敗するため .to(device) を統一使用） ---
bert_scml = bert_scml.to(device)

# 推論時は Dropout を止めたいので eval()、学習開始時は train() を呼ぶ。
# bert_scml.eval()   # 例：推論前
# bert_scml.train()  # 例：学習ループ開始時

# （参考）出力の扱い：
# - forward は {'logits': Tensor[B, C], 'loss': Tensor[]} 相当を返す簡易オブジェクト。
# - 学習時は labels=[B, C] を与えると BCEWithLogitsLoss を内部計算し output.loss に入る。

[info] using device = mps


In [5]:
# 7-6（説明コメント付き：マルチラベル推論・しきい値判定・サブセット精度）
import torch

# 入力テキスト：各サンプルに複数ラベルが立ち得る（マルチラベル）
text_list = ["今日の仕事はうまくいったが、体調があまり良くない。", "昨日は楽しかった。"]

# 教師ラベル（shape=[B, C]）：
#  - マルチラベルなので各列（ラベル）を独立に {0,1} で表す
#  - 例：C=2 のとき [仕事/体調] のような2軸を同時に予測するイメージ
labels_list = [[1, 1], [0, 1]]

# --- デバイス整合：モデル(bert_scml)の実デバイスに入力を合わせる ---
device = next(bert_scml.parameters()).device

# データの符号化
#  - padding='longest' はバッチ内の最長系列に合わせて動的にPADを付与
#  - 実運用では max_length+truncation=True+padding='max_length' で固定長化すると計算が安定
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")
encoding = {k: v.to(device) for k, v in encoding.items()}

# 教師ラベルテンソル
#  - 学習で BCEWithLogitsLoss を使う場合は float（{0.0,1.0}）が望ましい
#  - ここでは評価用に int 版も用意（比較演算時に便利）
labels_float = torch.tensor(labels_list, dtype=torch.float32, device=device)
labels_int = labels_float.to(torch.int64)

# --- 推論：本モデルは logits（活性前スコア）を返す ---
with torch.no_grad():
    output = bert_scml(**encoding)
scores = output.logits  # shape: [B, C]，各ラベルのロジット

# --- 判定（しきい値） ---
# 方式A：確率で判定（推奨。しきい値を柔軟に最適化できる）
probs = scores.sigmoid()  # p(y_c=1|x)
threshold = 0.5  # 必要に応じて ROC/PR で最適化
labels_predicted = (probs >= threshold).to(torch.int64)

# 方式B：ロジットの符号で判定（sigmoid≥0.5 と等価だが明示性に欠ける）
# labels_predicted = (scores > 0).to(torch.int64)

# --- 精度計算（subset accuracy：全ラベル一致率） ---
#  - サンプルごとに全てのラベルが正解しているかを判定し、その平均をとる厳しめの指標
num_correct = (labels_predicted == labels_int).all(dim=-1).sum().item()
accuracy = num_correct / labels_int.size(0)

# 参考：他の評価指標（マルチラベルで一般的）
#  - example-based F1（サンプル単位のF1を平均）
#  - micro/macro-F1（ラベル軸で集計）
#  - Hamming loss（ラベル単位の誤り率）
# これらは torchmetrics（MultilabelF1Score 等）で容易に算出可能

print("# scores (logits):", scores)
print("# probs (sigmoid):", probs)
print("# predicted labels:", labels_predicted)
print("# subset accuracy:", accuracy)

# scores (logits): tensor([[0.2937, 0.1129],
        [0.2349, 0.0266]], device='mps:0')
# probs (sigmoid): tensor([[0.5729, 0.5282],
        [0.5585, 0.5066]], device='mps:0')
# predicted labels: tensor([[1, 1],
        [1, 1]], device='mps:0')
# subset accuracy: 0.5


In [6]:
# 7-7（説明コメント付き：マルチラベル損失（BCEWithLogitsLoss）の計算）
# -----------------------------------------------------------------------------
# 目的：
#  - トークナイズ結果にマルチラベル教師（[B, C]）を同梱し、モデルの forward で
#    BCEWithLogitsLoss（sigmoid + BCE の数値安定版）を自動計算させて loss を得る。
#
# 理論メモ：
#  - マルチラベルでは各クラスを独立なベルヌーイとみなし、出力はロジット（活性前）。
#    教師は float 型の {0.0, 1.0} 行列（shape=[B, C]）。softmax は使わない。
#  - BCEWithLogitsLoss は内部で sigmoid を合成するため、出力に sigmoid をかけずに
#    生のロジットをそのまま渡すのが正しい。
#  - クラス不均衡が強い場合は pos_weight（shape=[C]）を指定すると陽性側の損失寄与を補正できる。
# -----------------------------------------------------------------------------

import torch

# --- デバイス整合：モデル(bert_scml)の実デバイスに入力を合わせる ---
device = next(bert_scml.parameters()).device

# --- データの符号化（バッチ内の最長に合わせて動的PAD） ---
# 実運用では max_length+truncation=True+padding="max_length" で固定長化すると計算が安定。
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")
encoding = {k: v.to(device) for k, v in encoding.items()}

# --- ラベルを同梱（[B, C]、float 型） ---
# BCEWithLogitsLoss は float を前提。int のままでも内部で float 化されるが明示が安全。
labels = torch.tensor(labels_list, dtype=torch.float32, device=device)
encoding["labels"] = labels

# --- 損失の計算 ---
# 学習時は train() にして Dropout を有効、評価時は eval() + no_grad()。
bert_scml.train()  # 例：学習フェーズを想定（評価なら bert_scml.eval() と no_grad() を併用）
output = bert_scml(**encoding)
loss = output.loss  # BCEWithLogitsLoss（バッチ平均）

print("# loss:", float(loss.detach().cpu()))

# （参考）不均衡対策：pos_weight を使う例（実装差し替え時）
#   C = labels.size(1)
#   pos_weight = torch.tensor([...], dtype=torch.float32, device=device)  # shape=[C]
#   criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
#   loss = criterion(output.logits, labels)

# loss: 0.671375036239624


In [8]:
# 7-9
data = json.load(open("chABSA-dataset/e00030_ann.json"))
print(data["sentences"][0])

{'sentence_id': 0, 'sentence': '当期におけるわが国経済は、景気は緩やかな回復基調が続き、設備投資の持ち直し等を背景に企業収益は改善しているものの、海外では、資源国等を中心に不透明な状況が続き、為替が急激に変動するなど、依然として先行きが見通せない状況で推移した', 'opinions': [{'target': 'わが国経済', 'category': 'OOD#general', 'polarity': 'neutral', 'from': 6, 'to': 11}, {'target': '景気', 'category': 'OOD#general', 'polarity': 'positive', 'from': 13, 'to': 15}, {'target': '設備投資', 'category': 'OOD#general', 'polarity': 'positive', 'from': 28, 'to': 32}, {'target': '企業収益', 'category': 'OOD#general', 'polarity': 'positive', 'from': 42, 'to': 46}, {'target': '資源国等', 'category': 'OOD#general', 'polarity': 'neutral', 'from': 62, 'to': 66}, {'target': '為替', 'category': 'OOD#general', 'polarity': 'negative', 'from': 80, 'to': 82}]}


In [9]:
# 7-10（説明コメント付き：chABSA からマルチラベル（negative/neutral/positive）データセットを構築）
# --------------------------------------------------------------------------------
# 目的：
#  - chABSA の JSON 群（各ファイルに sentences が入っている）から、
#    文テキストと極性ラベル（3値：negative/neutral/positive）を **マルチラベル** 形式で取り出す。
#    * ABSA は 1 文中に複数の「意見（aspect-opinion）」が存在し得るため、
#      文レベルでは negative/neutral/positive が**同時に立つ**可能性を想定（= マルチラベル）。
#
# 理論メモ：
#  - ここで作る labels は長さ 3 の one-hot ではなく **multi-hot**（例： [1,0,1]）。
#    後段の学習では BCEWithLogitsLoss（独立ベルヌーイ）を用いるのが標準。
#  - マルチクラス（排他的 1 ラベル）で扱いたい場合は、「優先順位ルール」や
#    「確率最大クラスを1つ選ぶ」などの単一化手続きを別途設ける。
# --------------------------------------------------------------------------------

import glob
import json
from collections import Counter

# 極性 → ラベルID の対応（ID順は学習・評価で再利用するため**固定**しておく）
category_id = {"negative": 0, "neutral": 1, "positive": 2}
id_to_category = {v: k for k, v in category_id.items()}  # デバッグ・可視化用の逆引き

dataset = []  # 出力：[{ 'text': <str>, 'labels': [neg, neu, pos] }, ...] の配列

# chABSA の JSON を列挙（ソートしておくと再現性が増す）
for file in sorted(glob.glob("chABSA-dataset/*.json")):
    # JSON をロード（UTF-8 を想定）
    with open(file, mode="r", encoding="utf-8") as f:
        data = json.load(f)

    # 各ファイルは {"sentences": [...]} の構造を想定
    # sentence 例：
    #   {
    #     "sentence": "・・・",
    #     "opinions": [
    #        {"polarity": "positive", ...},
    #        {"polarity": "negative", ...},
    #        ...
    #     ]
    #   }
    for sentence in data.get("sentences", []):
        text = sentence.get("sentence", "")

        # 文レベルのマルチラベルベクトル（[neg, neu, pos]）
        # - 同一文に複数 opinion がある場合、それぞれの極性に 1 を立てる（重複は 1 のまま）
        labels = [0, 0, 0]
        for opinion in sentence.get("opinions", []):
            pol = opinion.get("polarity")
            if pol in category_id:
                labels[category_id[pol]] = 1
            else:
                # 未知の極性が来た場合の保険（データ品質に依存）
                # 実務ではログ出力・スキップ数カウント等の監視を入れるとよい
                pass

        # 1 サンプル（テキスト + マルチラベル）を追加
        sample = {"text": text, "labels": labels}
        dataset.append(sample)

# ------------------（任意）簡易サニティチェックと統計------------------
# ラベル出現数（クラス別の陽性総数）を集計してみる
label_totals = Counter()
for s in dataset:
    for i, bit in enumerate(s["labels"]):
        if bit:
            label_totals[id_to_category[i]] += 1

print(f"# samples: {len(dataset)}")
print("# positives per class:", dict(label_totals))
print("# example:", dataset[0] if dataset else None)

# ここで得られた `dataset` は、後段の符号化フェーズで
# tokenizer(text, ...) → input_ids/attention_mask/... を作り、
# labels を torch.tensor([B, 3]) の float 型（{0.0,1.0}）にして BCEWithLogitsLoss へ渡す想定。

# samples: 6119
# positives per class: {'neutral': 230, 'positive': 2210, 'negative': 1746}
# example: {'text': '当連結会計年度におけるわが国経済は、政府の経済政策や日銀の金融緩和策により、企業業績、雇用・所得環境は改善し、景気も緩やかな回復基調のうちに推移いたしましたが、中国をはじめとするアジア新興国経済の減速懸念や、英国の欧州連合（ＥＵ）離脱決定、米国新政権への移行など、引き続き先行きは不透明な状況となっております', 'labels': [0, 1, 1]}


In [10]:
# 7-11
print(dataset[0])

{'text': '当連結会計年度におけるわが国経済は、政府の経済政策や日銀の金融緩和策により、企業業績、雇用・所得環境は改善し、景気も緩やかな回復基調のうちに推移いたしましたが、中国をはじめとするアジア新興国経済の減速懸念や、英国の欧州連合（ＥＵ）離脱決定、米国新政権への移行など、引き続き先行きは不透明な状況となっております', 'labels': [0, 1, 1]}
